In [ ]:
!pip install hdbscan --no-build-isolation --no-binary :all: --quiet   # extremely important to built it this way. Without it nothing works
!pip install bertopic --quiet

In [ ]:
from bertopic import BERTopic
import re
import pickle
import numpy as np
import pickle

In [ ]:
# reading files
categories = list()
labels_all = list()
scores = list()
with open('../input/hyppr-images-mapping/categories.txt', 'r') as file:
    for line in file.readlines():
        categories.append(line.lower().rstrip().split(','))
        
with open('../input/hyppr-images-mapping/labels.txt', 'r') as file:
    for line in file.readlines():
        labels_all.append(line.rstrip(',\n').split(','))

with open('../input/hyppr-images-mapping/scores.txt', 'r') as file:
    for line in file.readlines():
        scores.append(line.rstrip(',\n').split(','))
with open("../input/hyppr-images-mapping/all_ids.p", "rb") as f:
    all_ids = pickle.load(f)

In [ ]:
def set_key(dictionary, key, value):
    if key not in dictionary:
         dictionary[key] = value
    elif type(dictionary[key]) == list:
         dictionary[key].append(value)
    else:
         dictionary[key] = [dictionary[key], value]

In [ ]:
labels = [lab for label in labels_all for lab in label]
labels = list(set(labels))

In [ ]:
# splitting words by whitespace and hyphen (' ', '-')
labels_n = [" ".join(my_list) for my_list in labels_all]
labels_n = [" ".join(re.split(' |-', my_list)) for my_list in labels_n]

In [ ]:
bertopic = BERTopic(nr_topics = 200)
%time topics_bert, _ = bertopic.fit_transform(labels_n)

In [ ]:
bertopic.get_topic_freq().head()

In [ ]:
sum(bertopic.get_topic_freq()['Count'])

In [ ]:
bertopic.visualize_topics()

In [ ]:
num = 0
for topic in topics_bert:
    if topic == 245:
        num += 1
print(num)  

In [ ]:
objid_topic = dict()
for idOfTopic in range(len(topics_bert)):   # goin on list which contains all of the topics for all elements
    if topics_bert[idOfTopic] != -1:
        set_key(objid_topic, all_ids[idOfTopic], topics_bert[idOfTopic])
# reverse
topic_objids = dict()
for objid, topic in objid_topic.items():
    set_key(topic_objids, topic, objid)

In [ ]:
bertopic.get_topic(objid_topic['693df129-3d61-4692-b014-8ab3a02946ea'])

In [ ]:
nodata = ['dance', 'entertainment', 'tech']  # we don't have data on cite for this categories so we can't consider data communicated with it
cats = [cat[0] for cat in categories if cat[0] not in nodata]   # so we just throw it away
cats.append('none')

In [ ]:
similar_topics,similarity =  bertopic.find_topics('fashion', top_n=5)
similar_topics

In [ ]:
with open("../input/hyppr-images-mapping/objid_postid.p", "rb") as f:
    objid_postid = pickle.load(f)
# reverse:
postid_objid = dict()
for objid, postid in objid_postid.copy().items():
    set_key(postid_objid, postid, objid)

In [ ]:
with open("../input/hyppr-images-mapping/postid_objurls.p", "rb") as f:
    postid_objurls = pickle.load(f)
with open('../input/hyppr-images-mapping/category_to_posts_vision.p', 'rb') as f:   # opening given model
    category_to_posts = pickle.load(f)
category_to_posts = dict(category_to_posts)

posts_to_category = dict()
for cat, posts in category_to_posts.copy().items():
    for post in posts:
        if post not in posts_to_category.copy().keys():
            set_key(posts_to_category, post, cat)

In [ ]:
with open("../input/hyppr-images-mapping/objid_mark_category.p", "rb") as f:
       objid_mark_category = pickle.load(f)
with open("../input/hyppr-images-mapping/yRight.p", "rb") as f:
       yRight = pickle.load(f)

In [ ]:
threshold_for_model = 0.0
top_n = 15
category_topics = dict()
for cat in cats[:-1]:
    similar_topics, similarity = bertopic.find_topics(cat, top_n=top_n)
    if -1 in similar_topics:
        del similarity[similar_topics.index(-1)]
        similar_topics.remove(-1)
    
    similarity = [sim for sim in similarity if sim > threshold_for_model]
    similar_topics = [simT for simT in similar_topics[:len(similarity)]]
    set_key(category_topics, cat, similar_topics)

In [ ]:
for topic, objids in topic_objids.copy().items():
    if type(objids) != list:
        print('there is LISTS so you need to handle it in redefining set_key function')

In [ ]:
def set_key_redef(dictionary, key, value):   # redefining it for raveling lists in topic_objids 
    if key not in dictionary:
         dictionary[key] = value
    elif type(dictionary[key]) == list:
        temp_list = dictionary[key].copy()
        temp_list.extend(value)
        dictionary[key] = temp_list   # concatenates two lists

In [ ]:
topic_objids_copy = topic_objids.copy()
category_objids = dict()
objid_category = dict() # reverse dict
for category, topics in category_topics.copy().items():
    for topic in topics:
        set_key_redef(category_objids, category, topic_objids_copy[topic])   # neeed to unravel it for list structure
for category, objids in category_objids.copy().items():
    for objid in objids:
        objid_category[objid] = category

In [ ]:
for category, objids in category_objids.copy().items():
    if '693df129-3d61-4692-b014-8ab3a02946ea' in objids:
        print('there is no list file')

In [ ]:
yDef,yNew = [], []
no_post, no_topic, no_labels = 0, 0, 0
for post, cat in posts_to_category.copy().items():
    no_data = False
    if post in postid_objid.copy().keys():        
        objid = postid_objid[post]
        if objid in objid_category.copy().keys():
            cat_from_new_model = objid_category[objid]
            yNew.append(cat_from_new_model)
        else:
            yNew.append('none')
            no_topic += 1
    else:
        no_post += 1
        no_data = True
        
    if no_data is False:
        objid = postid_objid[post]
        if objid in objid_mark_category.copy().keys():
            yDef.append(objid_mark_category[objid])
        elif type(cat) is list:
            yDef.append(cat[0])
            no_labels +=1
        else:
            yDef.append(cat)
            no_labels +=1
print('There is {} post missing'.format(no_post))
print('There is {} obj to topic missing'.format(no_topic))
print(no_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score

conf_matr = confusion_matrix(yDef, yNew, labels = cats)

df_cm = pd.DataFrame(conf_matr, index = [i for i in range(len(cats))],
                  columns = [i for i in range(len(cats))])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm,cmap="YlGnBu",linewidths=1, annot=True, fmt = 'd')
plt.savefig('bertopicQualitynothresh.png', bbox_inches = 'tight')

In [ ]:
thresholds = np.arange(0,1,0.1)
f_1_scores_new, prec_scores, rec_scores = [], [], []

for threshold_for_model in thresholds:
    top_n = 10
    category_topics = dict()
    for cat in cats[:-1]:
        similar_topics, similarity = bertopic.find_topics(cat, top_n=top_n)
        if -1 in similar_topics:
            del similarity[similar_topics.index(-1)]
            similar_topics.remove(-1)

        similarity = [sim for sim in similarity if sim > threshold_for_model]
        similar_topics = [simT for simT in similar_topics[:len(similarity)]]
        set_key(category_topics, cat, similar_topics)

    topic_objids_copy = topic_objids.copy()
    category_objids = dict()
    objid_category = dict() # reverse dict
    for category, topics in category_topics.copy().items():
        for topic in topics:
            set_key_redef(category_objids, category, topic_objids_copy[topic])   # neeed to unravel it for list structure
    for category, objids in category_objids.copy().items():
        for objid in objids:
            objid_category[objid] = category
    
    yDef,yNew = [], []
    no_post, no_topic, no_labels = 0, 0, 0
    for post, cat in posts_to_category.copy().items():
        no_data = False
        if post in postid_objid.copy().keys():        
            objid = postid_objid[post]
            if objid in objid_category.copy().keys():
                cat_from_new_model = objid_category[objid]
                yNew.append(cat_from_new_model)
            else:
                yNew.append('none')
                no_topic += 1
        else:
            no_post += 1
            no_data = True

        if no_data is False:
            objid = postid_objid[post]
            if objid in objid_mark_category.copy().keys():
                yDef.append(objid_mark_category[objid])
            elif type(cat) is list:
                yDef.append(cat[0])
                no_labels +=1
            else:
                yDef.append(cat)
                no_labels +=1
    prec_scores.append(precision_score(yDef,yNew, labels = cats, average = 'macro'))
    rec_scores.append(recall_score(yDef,yNew, labels = cats, average = 'macro'))
    f_1_scores_new.append(f1_score(yDef,yNew, labels = cats, average = 'macro'))

In [ ]:
fig, ax = plt.subplots(1, 3, sharey = True, figsize = (16, 7))
ax[0].plot(thresholds, prec_scores, 'yx-')
ax[0].set_xlabel('t')
ax[0].set_ylabel('precision')
ax[1].plot(thresholds, rec_scores, 'rx-')
ax[1].set_xlabel('t')
ax[1].set_ylabel('recall')
ax[2].plot(thresholds, f_1_scores_new, 'gx-')
ax[2].set_xlabel('t')
ax[2].set_ylabel('$F$-score')
plt.savefig('bertopicQual.png', bbox_inches='tight')